# Setup

In [ ]:
# additional google colab setup
import sys


def colab_install():
    pass


if "google.colab" in sys.modules:
    print("Running in Google Colab")
    # Install required packages
    colab_install()
else:
    print("Not running in Google Colab")